In [142]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

URL = 'https://kolesa.kz/cars/'
HEADERS = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
          'accept':'*/*'}

data = {'name':[],'price':[],'text':[],'link':[]}
def get_html(url, params = None):
    r = requests.get(url,headers = HEADERS,params = params)
    return r

    


def get_content(html):
    soup = BeautifulSoup(html,'html.parser')
    items = soup.find_all('div',class_ ='a-info-side col-right-list')
    #print(items)
    for item in items:
        data['name'].append(item.find('a',class_ ="list-link ddl_product_link").get_text())
        data['price'].append(item.find('span',class_ ="price").get_text())
        data['text'].append(item.find('div',class_ ="a-search-description").get_text())
        #data['year'].append( l.split(',')[0]
        data['link'].append(item.find('a',class_ ="list-link ddl_product_link").get('href'))
        #link = 'https://kolesa.kz/' + link
        #data.append(URL_domen+i)
    
        
def parse():
    #300 страниц
    for i in range(1,300):
        html = get_html(URL + '?page='+str(i))

        if html.status_code == 200:

            get_content(html.text)

        else:
            print('Error: 1')
        
        
parse()

In [143]:
df = pd.DataFrame(data)
df.head()


,name,price,text,link
0,Renault Logan,\n 2 100 000 ₸,"\n 2007 г., Б/у седан, ...",/a/show/125208445
1,Mercedes-Benz Sprinter,\n 3 000 000 ₸,"\n 1996 г., Б/у микроав...",/a/show/125654667
2,Ford Scorpio,\n 480 000 ₸,"\n 1995 г., Б/у седан, ...",/a/show/125654494
3,Nissan Qashqai SE 2.0 CVT 4WD,\n 11 889 000 ₸,"\n 2021 г., новый кросс...",/a/show/112653384
4,Volkswagen Golf,\n 1 100 000 ₸,"\n 1993 г., Б/у хэтчбек...",/a/show/125654917


In [144]:
df.link = 'https://kolesa.kz/' + df.link
df['id'] = [i[-1] for i in df.link.str.split('/').values]
df['year'] = [re.findall(r'\d+',i[0])[0] for i in df.text.str.split(',').values]
df['quality'] = [i[1].split(' ')[1] for i in df.text.str.split(',').values]
df['type'] = [i[1].split(' ')[2] for i in df.text.str.split(',').values]
df['volume_l'] = [i[2].split(' ')[1] for i in df.text.str.split(',').values]
df['fuel_type'] = [i[3] for i in df.text.str.split(',').values]
df['Kpp'] = [i[4] for i in df.text.str.split(',').values]
df['price'] = [''.join(i) for i in df.price.str.findall(r'\d+')]

df = df[['id','name','price','year','quality','type','volume_l','fuel_type','Kpp','price','text','link']]

df.head()

,name,price,text,link,id,year,quality,type,volume_l,fuel_type,Kpp
0,Renault Logan,2100000,"\n 2007 г., Б/у седан, ...",https://kolesa.kz//a/show/125208445,125208445,2007,Б/у,седан,1.6,бензин,КПП механика
1,Mercedes-Benz Sprinter,3000000,"\n 1996 г., Б/у микроав...",https://kolesa.kz//a/show/125654667,125654667,1996,Б/у,микроавтобус,2.2,дизель,КПП механика
2,Ford Scorpio,480000,"\n 1995 г., Б/у седан, ...",https://kolesa.kz//a/show/125654494,125654494,1995,Б/у,седан,2,бензин,КПП автомат
3,Nissan Qashqai SE 2.0 CVT 4WD,11889000,"\n 2021 г., новый кросс...",https://kolesa.kz//a/show/112653384,112653384,2021,новый,кроссовер,2,бензин,КПП вариатор\n
4,Volkswagen Golf,1100000,"\n 1993 г., Б/у хэтчбек...",https://kolesa.kz//a/show/125654917,125654917,1993,Б/у,хэтчбек,1.6,бензин,КПП механика


In [147]:
df

,id,name,price,year,quality,type,volume_l,fuel_type,Kpp,price,text,link
0,125208445,Renault Logan,2100000,2007,Б/у,седан,1.6,бензин,КПП механика,2100000,"\n 2007 г., Б/у седан, ...",https://kolesa.kz//a/show/125208445
1,125654667,Mercedes-Benz Sprinter,3000000,1996,Б/у,микроавтобус,2.2,дизель,КПП механика,3000000,"\n 1996 г., Б/у микроав...",https://kolesa.kz//a/show/125654667
2,125654494,Ford Scorpio,480000,1995,Б/у,седан,2,бензин,КПП автомат,480000,"\n 1995 г., Б/у седан, ...",https://kolesa.kz//a/show/125654494
3,112653384,Nissan Qashqai SE 2.0 CVT 4WD,11889000,2021,новый,кроссовер,2,бензин,КПП вариатор\n,11889000,"\n 2021 г., новый кросс...",https://kolesa.kz//a/show/112653384
4,125654917,Volkswagen Golf,1100000,1993,Б/у,хэтчбек,1.6,бензин,КПП механика,1100000,"\n 1993 г., Б/у хэтчбек...",https://kolesa.kz//a/show/125654917
...,...,...,...,...,...,...,...,...,...,...,...,...
5975,123541915,ВАЗ (Lada) 2112 (хэтчбек),795000,2006,Б/у,хэтчбек,1.6,бензин,КПП механика,795000,"\n 2006 г., Б/у хэтчбек...",https://kolesa.kz//a/show/123541915
5976,125633369,Toyota Mark X,2800000,2006,Б/у,седан,2.5,бензин,КПП автомат,2800000,"\n 2006 г., Б/у седан, ...",https://kolesa.kz//a/show/125633369
5977,125633229,Audi A6 allroad,4900000,2007,Б/у,универсал,3.2,бензин,КПП автомат,4900000,"\n 2007 г., Б/у универс...",https://kolesa.kz//a/show/125633229
5978,125231616,Mercedes-Benz E 320,4800000,2002,Б/у,седан,3.2,бензин,КПП автомат,4800000,"\n 2002 г., Б/у седан, ...",https://kolesa.kz//a/show/125231616
